In [0]:
import numpy as np
import pandas as pd

df1 = pd.read_csv('train_genba.tsv', sep='\t')
df2 = pd.read_csv('train_goto.tsv', sep='\t')
df = pd.merge(df1, df2, on='pj_no', how='outer')

del df1
del df2

dftest1 = pd.read_csv('test_genba.tsv', sep='\t')
dftest2 = pd.read_csv('test_goto.tsv', sep='\t')
dftest = pd.merge(dftest1, dftest2, on='pj_no', how='outer')

del dftest1
del dftest2

df = df.reset_index(drop=False)
dftest = dftest.reset_index(drop=False)

print(len(df))
df.head()

6461


,index,pj_no,jukyo,chiseki_js_hb,chiseki_kb_hb,yoto1,yoto2,kempei1,kempei2,yoseki1,yoseki2,josui,gesui,gas,usui,tateuri_su,tochiuri_su,joken_su,hy1f_date_su,hy2f_date_su,hy3f_date_su,road1_hk,road1_sb,road1_fi,road1_mg,road2_hk,road2_sb,road2_fi,road2_mg,road3_hk,road3_sb,road3_fi,road3_mg,road4_hk,road4_sb,road4_fi,road4_mg,kaoku_um,kaoku_hb,yheki_kotei,...,rs_s_m_ari,rs_s_m_nashi,rs_s_tahata,rs_n_kdate2,rs_n_kdate3,rs_n_parking,rs_n_zoki,rs_n_m_ari,rs_n_m_nashi,rs_n_tahata,rosen_nm1,eki_nm1,bas_toho1,eki_kyori1,bastei_nm1,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2,id,keiyaku_pr,tc_mseki,tt_mseki,levelplan,fukuin,road_st,magutchi,setsudo_hi,setsudo_kj,jigata,hiatari,niwasaki,garage,kobetsu1,kobetsu2,kobetsu3,kobetsu4
0,0,0,埼玉県朝霞市泉水,109.26,109.10,工業地域,NaN,60,0.0,200,0.0,公営,公共下水,個別プロパン,宅内処理,1.0,0.0,0.0,0.0,1.0,0.0,北,私道,4.0,7.0,東,私道,4.0,10.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,（無）,0.0,0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,東武東上線,朝霞台,徒歩,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_0000,39800000,109.26,104.43,2F/4LDK,4.0,問題なし,9.9,東,良い,整形地,普通,4.0,1,床暖房付,NaN,NaN,NaN
1,1,1,埼玉県川越市大字的場,136.15,136.00,第一種低層住居専用地域,NaN,60,0.0,200,0.0,公営,公共下水,個別プロパン,宅内処理,1.0,0.0,0.0,0.0,1.0,0.0,北西,公道,4.2,9.3,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,（無）,0.0,0.0,...,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,JR川越線,的場,徒歩,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_0001,22300000,136.11,105.16,2F/4LDK,4.2,問題なし,9.3,北西,普通,不整形地,普通,3.0,1,NaN,NaN,NaN,NaN
2,2,2,埼玉県川口市安行原,86.92,86.52,第一種低層住居専用地域,NaN,50,0.0,100,0.0,私営,個別浄化槽,個別プロパン,宅内処理,1.0,0.0,0.0,0.0,1.0,0.0,東,私道,4.0,11.1,北,公道,3.8,5.2,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,（有）,62.1,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,埼玉高速鉄道線,新井宿,徒歩,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_0002,19800000,87.07,85.94,2F/4LDK,4.0,問題なし,11.1,東,普通,整形地,普通,2.8,1,NaN,NaN,NaN,NaN
3,3,3,埼玉県草加市松江,164.91,164.07,工業地域,NaN,60,0.0,200,0.0,公営,公共下水,都市ガス,宅内処理,1.0,0.0,0.0,0.0,1.0,0.0,南,公道,4.0,13.3,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,（無）,0.0,0.3,...,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,東武伊勢崎線,獨協大学前駅〈草加松原〉,徒歩,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_0003,33990000,163.75,105.98,2F/4LDK,4.0,問題なし,10.5,南,普通,整形地,良い,4.0,2,NaN,NaN,NaN,NaN
4,4,4,埼玉県新座市野火止,403.58,403.58,第一種低層住居専用地域,NaN,50,0.0,80,0.0,公営,公共下水,都市ガス,宅内処理,3.0,0.0,0.0,0.0,3.0,0.0,南西,私道,4.0,13.9,北東,私道,4.0,13.9,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,（無）,0.0,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JR武蔵野線,新座,徒歩,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_0004,30800000,111.51,89.01,2F/4LDK,4.0,問題なし,13.5,北,良い,整形地,普通,2.0,2,床暖房付,NaN,NaN,NaN


In [0]:
dftest_id = dftest['id']
dftest_id.head()

0    test_0000
1    test_0001
2    test_0002
3    test_0003
4    test_0004
Name: id, dtype: object

In [0]:
# 欠損値をペアワイズ法により削除
for i, j in zip(df.isnull().any(), df.columns):
    if i == True:
        df = df.drop(j, axis=1)

for i, j in zip(dftest.isnull().any(), dftest.columns):
    if i == True:
        dftest = dftest.drop(j, axis=1)

df = df.drop('road1_sb', axis=1)
df = df.drop('gk_yoc_tm', axis=1)

dftest = dftest.drop('chiseki_kb_hb', axis=1)
dftest = dftest.drop('tt_mseki_avg_hb', axis=1)

factorize = []

# string => float, int
for i, j in zip(df.columns, df.dtypes):
    if j == 'object':
        factorize.append(i)
    
for i in factorize:
    id_1, id_2 = df[i].factorize()
    id_3 = pd.DataFrame({i: id_1})
    #df = pd.concat(objs=[df, id_3], axis=0)
    df[i] = id_3
    dftest[i] = id_3
    
df.head()

,index,pj_no,jukyo,chiseki_js_hb,yoto1,kempei1,yoseki1,josui,gesui,gas,usui,road1_hk,road1_fi,road1_mg,kaoku_um,yheki_kotei,yheki_umu,yheki_yohi,gk_sho_kyori,gk_chu_kyori,kborjs,hw_status,toshikuiki1,kodochiku,chikukeikaku,keikakuroad,kaihatsukyoka,t53kyoka,hokakyoka,bokachiiki,kinshijiko,rosenka_hb,koji_hb,kijun_hb,mseki_yt_hb,tc_mseki_min_hb,tc_mseki_max_hb,tt_mseki_min_hb,tt_mseki_max_hb,tc_mseki_avg_hb,fi4m_yohi,fi3m_yohi,bus_yohi,gk_sho_tm,gk_chu_tm,rosen_nm1,eki_nm1,bas_toho1,eki_kyori1,id,keiyaku_pr,tc_mseki,tt_mseki,niwasaki,garage
0,0,0,0,109.26,0,60,200,0,0,0,0,0,4.0,7.0,0,0.6,0,0,800,1000,0,0,0,0,0,0,0,0,0,0,0,135000,188000,197000,109.26,109.26,109.26,104.43,104.43,109.26,0,0,0,10,13,0,0,0,17,0,39800000,109.26,104.43,4.0,1
1,1,1,1,136.15,1,60,200,0,0,0,0,1,4.2,9.3,0,0.0,0,0,1680,1840,0,0,0,1,0,0,0,0,0,1,0,68000,84900,48600,136.15,136.15,136.15,109.10,109.10,136.15,0,0,0,21,23,1,1,0,13,1,22300000,136.11,105.16,3.0,1
2,2,2,2,86.92,1,50,100,1,1,0,0,2,4.0,11.1,1,0.2,0,0,600,550,0,1,0,1,0,0,0,0,0,0,0,83000,116000,124000,86.92,86.92,86.92,85.96,85.96,86.92,0,0,1,8,7,2,2,0,27,2,19800000,87.07,85.94,2.8,1
3,3,3,3,164.91,0,60,200,0,0,1,0,3,4.0,13.3,0,0.3,0,0,1220,111,1,0,0,1,0,0,0,0,1,0,0,91000,122000,113000,164.07,164.07,164.07,105.99,105.99,164.07,0,0,0,16,2,3,3,0,18,3,33990000,163.75,105.98,4.0,2
4,4,4,4,403.58,1,50,80,0,0,1,0,4,4.0,13.9,0,0.1,0,0,480,2800,1,0,0,1,0,0,0,0,0,1,0,160000,179000,160000,334.58,111.52,111.53,89.27,89.27,111.52,0,0,0,6,35,4,4,0,7,4,30800000,111.51,89.01,2.0,2


In [0]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression # 線形回帰　# 重回帰
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xgb
import lightgbm as lgb

#clf = linear_model.LinearRegression() # 15.83164
#clf = LinearRegression() # 15.83164 
#clf = KNeighborsRegressor() # 16.44560
#clf = Ridge() # 15.87452
#clf = Lasso() # 14.65003 # 2019/06/20
#clf = ElasticNet(alpha = 0.5) # 12.74356
#clf = DecisionTreeRegressor(random_state = 0) # 16.63658
#clf = LinearSVR() # 25.29030
#clf = GradientBoostingRegressor() # 11.90509
#clf = BaggingRegressor() # 12.22117 # 2019/6/21
#clf = AdaBoostRegressor() # 15.48687
clf = xgb.XGBRegressor(eta=0.01, max_depth=10, subsumple=5)
#clf = lgb.LGBMClassifier()

# 説明変数に "quality (品質スコア以外すべて)" を利用
#wine_except_quality = wine.drop("quality", axis=1)
#X = wine_except_quality.as_matrix()

dfall = df.drop('keiyaku_pr', axis=1)
X = dfall.as_matrix()

# 目的変数に "quality (品質スコア)" を利用
Y = df['keiyaku_pr'].as_matrix()
 
# 予測モデルを作成
clf.fit(X, Y)
 
# 偏回帰係数
#print(pd.DataFrame({"Name":dfall.columns,
#                   "Coefficients":clf.coef_}).sort_values(by='Coefficients') )
 
# 切片 (誤差)
#print(clf.intercept_)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
#予測
XX = dftest.as_matrix()

YY = clf.predict(XX)

df_pre = pd.DataFrame(YY)
df_pre.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


,0
0,20500000
1,23500000
2,24800000
3,24800000
4,24800000


In [0]:
df_pre = df_pre.astype(int)
df_pre = df_pre.round()
df_pre = pd.concat(objs=[dftest_id, df_pre], axis=1)
df_pre.head()

,id,0
0,test_0000,20500000
1,test_0001,23500000
2,test_0002,24800000
3,test_0003,24800000
4,test_0004,24800000


In [0]:
df_pre.to_csv('test_pre.tsv', sep='\t', index=False, header=False)